**Sinergia Dev** playbook base

# Openai Responses API

- python install requirements

In [2]:
!pip install openai

In [3]:
import getpass
import os

# set env for openai key:

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [5]:
from openai import OpenAI
import pprint
client = OpenAI()

# GENERA UN TEXTO EN BASE A UN PROMPT SIMPLE 
# NIVEL 1

## ChatGPT LLM single call
response = client.responses.create(
    model="gpt-4o-mini",
    input="Escribe un cuento de buenas noches en una sola oración sobre un unicornio."
)

## print response output text
print("response.output_text:\n")
pprint.pprint(response.output_text)
print("\n\n")

## debug full response object
print("response object:\n")
pprint.pprint(response.model_dump())

response.output_text:

Una noche estrellada, un unicornio llamado Luno iluminó el bosque con su cuerno brillante mientras danzaba entre los sueños de los niños, asegurando que cada uno despertara con una sonrisa en el corazón.
response object:

{'created_at': 1745446012.0,
 'error': None,
 'id': 'resp_6809647cd4248192af35ade638330f1807e15694723ac3b9',
 'incomplete_details': None,
 'instructions': None,
 'max_output_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [{'content': [{'annotations': [],
                          'text': 'Una noche estrellada, un unicornio llamado '
                                  'Luno iluminó el bosque con su cuerno '
                                  'brillante mientras danzaba entre los sueños '
                                  'de los niños, asegurando que cada uno '
                                  'despertara con una sonrisa en el corazón.',
                          'type': 'output_text'}],
     

In [14]:
# GENERA UN TEXTO CON UN PROMPT SIMPLE PERO AGREGANDOLE VARIABLES ESTATICAS (INSTRUCCIONES)
# NIVEL 2

## ChatGPT LLM single call
response = client.responses.create(
    model="gpt-4o", # "gpt-4o-mini" no entiende las instrucciones...
    instructions="You are an english translator. User will write a prompt. Your goals is to translate the user prompt", # estas son variables estaticas tambien
    input="Escribe un cuento de buenas noches en una sola oración sobre un unicornio.",
)

## print response output text
print("response.output_text:\n")
pprint.pprint(response.output_text)
print("\n\n")

## debug full response object
print("response object:\n")
pprint.pprint(response.model_dump())

response.output_text:

Write a one-sentence bedtime story about a unicorn.



response object:

{'created_at': 1745447673.0,
 'error': None,
 'id': 'resp_68096af9513c81929ea75ee392a952750ba1855e4c491965',
 'incomplete_details': None,
 'instructions': 'You are an english translator. User will write a prompt. '
                 'Your goals is to translate the user prompt',
 'max_output_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-2024-08-06',
 'object': 'response',
 'output': [{'content': [{'annotations': [],
                          'text': 'Write a one-sentence bedtime story about a '
                                  'unicorn.',
                          'type': 'output_text'}],
             'id': 'msg_68096af9d09881928f3adc91d67b24190ba1855e4c491965',
             'role': 'assistant',
             'status': 'completed',
             'type': 'message'}],
 'parallel_tool_calls': True,
 'previous_response_id': None,
 'reasoning': {'effort': None, 'generate_summary': None, 'summary

In [18]:
# GENERA UN TEXTO CON UN history messages USANDO DIFERENTES ROLES (developer, user, assistant)
# MANEJO DEL CONTEXTO...

response = client.responses.create(
    model="gpt-4o-mini",
    input="contame un chiste de programadores",
)
print(response.output_text)
print("\n\n")
second_response = client.responses.create(
    model="gpt-4o-mini",
    previous_response_id=response.id, # esta es la forma de persistir el puntero del historial entre llamadas
    input=[{"role": "user", "content": "explicame porque es gracioso."}],
)
print(second_response.output_text)


print("\n\n")

## debug full response object
print("response object:\n")
pprint.pprint(response.model_dump())
print("\n\n")

## debug full response object
print("second_response object:\n")
pprint.pprint(second_response.model_dump())

¡Claro! Aquí tienes uno:

¿Por qué los programadores prefieren el café al té?

Porque el té tiene demasiados "bugs". ☕🐛

Espero que te haya sacado una sonrisa. ¡Si quieres otro, avísame!



¡Claro! El chiste juega con dos conceptos:

1. **Café vs. Té**: Es un estereotipo en la cultura de los programadores que muchos prefieren el café, especialmente por su cafeína, que los ayuda a mantenerse despiertos durante largas horas de codificación.

2. **Bugs**: En programación, un "bug" es un error o falla en el código. La broma implica que el té "tiene demasiados bugs", insinuando que podría ser menos efectivo o más problemático, como un programa lleno de errores.

La clave está en la conexión entre el término técnico ("bugs") y la idea de que el té es menos deseable. Es un juego de palabras que combina lo cotidiano con el mundo de la programación, lo que suele generar una sonrisa entre quienes están familiarizados con esos conceptos. ¡Espero que esto aclare la broma!



response object:

{'cr

# Structured Outputs

In [25]:
import json

# GENERA UN JSON EN BASE A UNA ESTRUCTURA json DADA
# NIVEL 2

# structured output models support by:

# gpt-4.5-preview-2025-02-27 and later
# o3-mini-2025-1-31 and later
# o1-2024-12-17 and later
# gpt-4o-mini-2024-07-18 and later
# gpt-4o-2024-08-06 and later

response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Matias and Jony are going to SinergIA-Dev on 24 thursday, april at 15hs."}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "calendar_event",
            "schema": {
                "type": "object",
                "properties": {
                    "event_name": {
                        "type": "string"
                    },
                    "date": {
                        "type": "string",
                    },
                    "participants": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    },
                },
                "required": ["event_name", "date", "participants"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

event = json.loads(response.output_text)

pprint.pprint(event)

{'event_name': 'SinergIA-Dev', 'date': '24 April 2023', 'participants': ['Matias', 'Jony']}


In [ ]:
import json
import pprint

# Structured Outputs for chain-of-thought math tutoring

response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "math_reasoning",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": { "type": "string" },
                                "output": { "type": "string" }
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": { "type": "string" }
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

math_reasoning = json.loads(response.output_text)
pprint.pprint(math_reasoning)


{'final_answer': 'x = -\\frac{15}{4}',
 'steps': [{'explanation': "Start by isolating the term with the variable 'x'. "
                           'We need to move the constant term on the left side '
                           'to the right side. To do this, subtract 7 from '
                           'both sides of the equation.',
            'output': '8x + 7 - 7 = -23 - 7'},
           {'explanation': 'This simplifies to:', 'output': '8x = -30'},
           {'explanation': "Next, solve for 'x' by dividing both sides of the "
                           'equation by 8.',
            'output': '\\( x = \\frac{-30}{8} \\)'},
           {'explanation': 'Simplify the fraction by dividing the numerator '
                           'and the denominator by their greatest common '
                           'divisor, which is 2.',
            'output': '\\( x = \\frac{-15}{4} \\)'},
           {'explanation': 'Thus, the solution to the equation is:',
            'output': 'x = -\\frac{15}{4